In [2]:
import numpy as np
import pandas as pd
import glob
from PIL import Image
import imageio
import imageio
import skimage
from tqdm import tqdm
import requests
import re
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import ast
import matplotlib.pyplot as plt
from zipfile import ZipFile
import os

# Giải nén ảnh
with ZipFile("Images.zip", 'r') as zip_ref:
    zip_ref.extractall("Images")

savelocation = 'Images/Images/'

In [3]:
df_movietotal = pd.read_csv('MovieGenre.csv', encoding='latin1')
# Loại các dòng không có thể loại
df_movietotal = df_movietotal.dropna(subset=["Genre"])

# Chuyển thể loại thành dạng: "Comedy|Action|..." → để giống notebook gốc
df_movietotal["genres"] = df_movietotal["Genre"].apply(lambda x: x.replace(", ", "|") + "|")

# Vì ảnh của bạn đặt tên theo số ID, ta cần chuyển 'imdbId' thành 'imdb_id' và kiểu str
df_movietotal["imdb_id"] = df_movietotal["imdbId"].astype(str)
print(df_movietotal.columns)
print(df_movietotal.shape)  # số dòng và cột
print(df_movietotal.head())  # in 5 dòng đầu xem dữ liệu

Index(['imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster',
       'genres', 'imdb_id'],
      dtype='object')
(39963, 8)
   imdbId                           Imdb Link  \
0  114709  http://www.imdb.com/title/tt114709   
1  113497  http://www.imdb.com/title/tt113497   
2  113228  http://www.imdb.com/title/tt113228   
3  114885  http://www.imdb.com/title/tt114885   
4  113041  http://www.imdb.com/title/tt113041   

                                Title  IMDB Score                       Genre  \
0                    Toy Story (1995)         8.3  Animation|Adventure|Comedy   
1                      Jumanji (1995)         6.9     Action|Adventure|Family   
2             Grumpier Old Men (1995)         6.6              Comedy|Romance   
3            Waiting to Exhale (1995)         5.7        Comedy|Drama|Romance   
4  Father of the Bride Part II (1995)         5.9       Comedy|Family|Romance   

                                              Poster  \
0  https://images-na.ssl-ima

In [4]:
df_movietotal = df_movietotal[~df_movietotal["genres"].str.contains("TV Movie")]
df_movietotal = df_movietotal[~df_movietotal["genres"].str.contains("Foreign")]

In [5]:
df_movietotal_copy = df_movietotal.copy()

In [6]:
len(df_movietotal)

39963

In [7]:
df_movietotal = df_movietotal.sample(frac=.1)
df_movietotal_copy = pd.concat([df_movietotal, df_movietotal_copy]).drop_duplicates(keep=False)

In [8]:
label_dict = {"word2idx": {}, "idx2word": []}
idx = 0
# genre_per_movie = df_movietotal["genres"].apply(lambda x: str(x).split("|")[:-1])
# Tách danh sách thể loại từ cột "genres"
genre_per_movie = df_movietotal["genres"].apply(lambda x: str(x).strip("|").split("|"))

for l in [g for d in genre_per_movie for g in d]:
    if l in label_dict["idx2word"]:
        pass
    else:
        label_dict["idx2word"].append(l)
        label_dict["word2idx"][l] = idx
        idx += 1
n_classes = len(label_dict["idx2word"])

In [9]:
def genre_count(df, label_dict):
    max_genre = 0
    for label in label_dict["idx2word"]:
        occurrences = len((df[df['genres'].str.contains(label)]))
        print(label, occurrences)
        if occurrences > max_genre:
            max_genre = occurrences
    return max_genre

In [10]:
max_genre = genre_count(df_movietotal, label_dict)

Drama 2052
Film-Noir 45
Thriller 503
Romance 595
Comedy 1224
Action 544
Family 195
Sci-Fi 224
Western 94
Horror 407
Documentary 389
Crime 510
War 119
Fantasy 202
Animation 162
Adventure 413
Biography 211
Mystery 241
Music 194
Musical 81
Short 93
Sport 69
History 137
News 14
Adult 1


In [22]:
# IMBALANCE: OVERSAMPLING SOLUTION
df_movietotal_copy = df_movietotal_copy[~df_movietotal_copy["genres"].str.contains("Comedy")]
df_movietotal_copy = df_movietotal_copy[~df_movietotal_copy["genres"].str.contains("Drama")]

for label in label_dict["idx2word"]:
    if label not in ["Drama", "Comedy"]:
        len_genre = len(df_movietotal[df_movietotal['genres'].str.contains(label)])
        df_genre = df_movietotal_copy[df_movietotal_copy['genres'].str.contains(label)]
        #df_genre['genres'] = [label+"|" for i in range (0, len(df_genre))]
        if (max_genre - len_genre) > 0:
            if len_genre > 3000:
                param = 0
            elif len_genre > 2000:
                param = 0.3
            elif len_genre > 1000:
                param = 0.5
            else:
                param = 0.9
            df_class_over = df_genre.sample(int((max_genre-len_genre)*param)+1, replace=True)
            df_movietotal = pd.concat([df_movietotal, df_class_over], axis=0)

print('Random over-sampling:')
print(genre_count(df_movietotal, label_dict))

Random over-sampling:
Drama 70
Film-Noir 1901
Thriller 3500
Romance 3194
Comedy 35
Action 4038
Family 2940
Sci-Fi 2247
Western 2095
Horror 3104
Documentary 8803
Crime 2606
War 1983
Fantasy 2628
Animation 2532
Adventure 3477
Biography 3168
Mystery 1872
Music 3600
Musical 1923
Short 2276
Sport 1899
History 2145
News 1855
Adult 1848
8803


In [23]:
len(df_movietotal)

26609

In [24]:
print(genre_count(df_movietotal, label_dict))

Drama 70
Film-Noir 1901
Thriller 3500
Romance 3194
Comedy 35
Action 4038
Family 2940
Sci-Fi 2247
Western 2095
Horror 3104
Documentary 8803
Crime 2606
War 1983
Fantasy 2628
Animation 2532
Adventure 3477
Biography 3168
Mystery 1872
Music 3600
Musical 1923
Short 2276
Sport 1899
History 2145
News 1855
Adult 1848
8803


In [25]:
image_glob = glob.glob(savelocation + "*.jpg")
img_dict = {}


def get_id(filename):
    return filename.split("/")[-1].split(".")[0]

In [26]:
for fn in image_glob:
    try:
        img_dict[get_id(fn)] = imageio.imread(fn)
    except:
        pass

<ipython-input-26-072d24dd8caa>:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_dict[get_id(fn)] = imageio.imread(fn)


In [27]:
print(len(image_glob))

997


In [28]:
def show_img(id):
    title = df_movietotal[df_movietotal["imdb_id"] == id]["original_title"].values[0]
    genre = df_movietotal[df_movietotal["imdb_id"] == id]["genres"].values[0]
    plt.imshow(img_dict[id])
    plt.title("{} \n {}".format(title, genre))

In [29]:
def preprocess(img, size=(150, 101, 3)):
    try:
        if img.ndim == 2:
            img = np.stack([img] * 3, axis=-1)
        elif img.ndim == 3 and img.shape[2] == 1:
            img = np.repeat(img, 3, axis=2)
        elif img.ndim == 3 and img.shape[2] > 3:
            img = img[:, :, :3]

        img = skimage.transform.resize(img, size, preserve_range=True, anti_aliasing=True)
        img = img.astype(np.float32)
        img = (img / 127.5) - 1.
        return img
    except Exception as e:
        print(f"Error preprocessing image: {e}")
        return None

In [30]:
def prepare_data(data, img_dict, label_dict, size=(150, 101, 3)):
    print("Generation dataset...")
    dataset = []
    y = []
    ids = []
    n_samples = len(img_dict)
    print("got {} posters".format(n_samples))
    for k in img_dict:
        if k in data["imdb_id"].values:
            G = data[data["imdb_id"] == k]["genres"].values
            for g in G:
                g = g.split("|")[:-1]
                img = preprocess(img_dict[k], size)
                if img is None or np.isnan(img).any():
                  continue
                if img is None or img.shape != size or np.isnan(img).any():
                  print(f"Skipping image {k} due to bad preprocessing. Original shape: {img_dict[k].shape}")
                  continue
                l = np.sum([np.eye(n_classes, dtype="uint8")[label_dict["word2idx"][s]]
                                                            for s in g], axis=0)
                y.append(l)
                dataset.append(img)
                ids.append(k)
    print("DONE")
    print(len(dataset))
    return dataset, y, ids

In [31]:
df_movietotal = df_movietotal[['genres', 'imdb_id', 'Title']]
df_movietotal = df_movietotal[df_movietotal['imdb_id'].isin(img_dict.keys())]

In [32]:
SIZE = (150, 101, 3)
dataset, y, ids =  prepare_data(df_movietotal, img_dict, label_dict, size=SIZE)

Generation dataset...
got 997 posters
DONE
739


In [33]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.metrics import BinaryAccuracy
from keras.layers import BatchNormalization
# from tensorflow import keras
# from tensorflow.keras import layers

In [52]:
# n = len(dataset) - 100
# n = 600
# n_test = 100

In [34]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(SIZE[0], SIZE[1], 3)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))
print("Số nhãn:", n_classes)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Số nhãn: 25


In [35]:
# # n = 400
# # n_test = 100
# if n >= len(dataset):
#     print("Chỉ số n vượt quá kích thước dataset!")
# else:
#     n_test = min(n_test, len(dataset) - n)
#     X_test = dataset[n:n + n_test]
#     y_test = y[n:n + n_test]
#     pred = model.predict(np.array(X_test))
#     print("X_test shape:", np.array(X_test).shape)
#     print("Length X_test:", len(X_test))
#     print("Dataset length:", len(dataset))
#     print("n:", n)
#     print("n + n_test:", n + n_test)

In [36]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adagrad(),
              metrics=[BinaryAccuracy()])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=0.2, random_state=42)

# Huấn luyện
model.fit(np.array(X_train), np.array(y_train), batch_size=16, epochs=100, validation_split=0.1)

# Đánh giá
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(f"Loss: {loss}, Accuracy: {accuracy}")


Epoch 1/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 99s 3s/step - binary_accuracy: 0.6836 - loss: 0.7965 - val_binary_accuracy: 0.4820 - val_loss: 0.7583
Epoch 2/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - binary_accuracy: 0.8104 - loss: 0.4617 - val_binary_accuracy: 0.5287 - val_loss: 0.7889
Epoch 3/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - binary_accuracy: 0.8272 - loss: 0.4078 - val_binary_accuracy: 0.5520 - val_loss: 0.7425
Epoch 4/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - binary_accuracy: 0.8404 - loss: 0.3820 - val_binary_accuracy: 0.6007 - val_loss: 0.6972
Epoch 5/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - binary_accuracy: 0.8480 - loss: 0.3637 - val_binary_accuracy: 0.6420 - val_loss: 0.6388
Epoch 6/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 86s 2s/step - binary_accuracy: 0.8632 - loss: 0.3523 - val_binary_accuracy: 0.7267 - val_loss: 0.5482
Epoch 7/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - binary_accuracy: 0.8596 - loss: 0.3382 - val_binary_accuracy: 0.7947 - val_loss: 0.4870
Epoch 8/100

In [ ]:
# print("X shape:", np.array(dataset[:n]).shape)
# print("Y shape:", np.array(y[:n]).shape)
# np.sum(y, axis=1)
# y = np.array(y).astype(np.float32)

In [ ]:
# model.fit(np.array(dataset[: n]), np.array(y[: n]), batch_size=16, epochs=100,
#           verbose=1, validation_split=0.1)

In [82]:
# def accuracy_score(y_test, pred):
#     value = 0
#     for i in range(0, len(pred)):
#         first3_index = np.argsort(pred[i])[-3:]
#         correct = np.where(y_test[i] == 1)[0]
#         for j in first3_index:
#             if j in correct:
#                 value += 1
#     print(value/len(pred))

In [40]:
# print("Dự đoán:", pred[0])
# print("Đáp án đúng:", y_test[0])
print(y_train[:5])
import numpy as np
from sklearn.metrics import f1_score

pred = model.predict(np.array(X_test))  # Đúng input
y_pred_bin = (pred >= 0.5).astype(int)

# print(y_test.shape)
# print(y_pred_bin.shape)
f1_micro = f1_score(y_test, y_pred_bin, average='micro')
f1_macro = f1_score(y_test, y_pred_bin, average='macro')

print(f"F1-score (micro): {f1_micro:.4f}")
print(f"F1-score (macro): {f1_macro:.4f}")

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=uint64), array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=uint64), array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=uint64), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=uint64), array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0], dtype=uint64)]
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step
F1-score (micro): 0.9187
F1-score (macro): 0.6951


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [41]:
def evaluate_multilabel(y_true, y_pred, k=3, threshold=0.5):
    n_samples = len(y_pred)
    top_k_correct = 0

    # Các giá trị để tính precision, recall, f1
    true_positives = 0
    predicted_positives = 0
    actual_positives = 0

    for i in range(n_samples):
        # Top-k accuracy
        top_k = np.argsort(y_pred[i])[-k:]
        true_labels = np.where(y_true[i] == 1)[0]
        if any(label in top_k for label in true_labels):
            top_k_correct += 1

        # Dự đoán theo ngưỡng
        predicted_labels = np.where(y_pred[i] >= threshold)[0]

        tp = len(set(predicted_labels) & set(true_labels))
        true_positives += tp
        predicted_positives += len(predicted_labels)
        actual_positives += len(true_labels)

    # Tránh chia cho 0
    precision = true_positives / predicted_positives if predicted_positives else 0
    recall = true_positives / actual_positives if actual_positives else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
    top_k_accuracy = top_k_correct / n_samples

    print(f"Top-{k} Accuracy: {top_k_accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")

In [42]:
# accuracy_score(y_test, pred)
evaluate_multilabel(y_test, pred, k=3, threshold=0.5)

Top-3 Accuracy: 0.9217
Precision: 0.9800
Recall:    0.8647
F1-score:  0.9188


In [ ]:
model.save("my_model.keras")
from google.colab import files
files.download("my_model.keras")

history = model.fit(np.array(X_train), np.array(y_train), batch_size=16, epochs=100, validation_split=0.1)
print("Lịch sử:", history)

In [ ]:
import matplotlib.pyplot as plt

# Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Dự đoán
y_pred = model.predict(X_test)
y_pred_bin = (y_pred >= 0.5).astype(int)

# Precision, Recall, F1 per label
precision = precision_score(y_test, y_pred_bin, average=None)
recall = recall_score(y_test, y_pred_bin, average=None)
f1 = f1_score(y_test, y_pred_bin, average=None)

# Vẽ biểu đồ
labels = [f'Label {i}' for i in range(len(precision))]

x = np.arange(len(labels))
width = 0.3

plt.figure(figsize=(15, 6))
plt.bar(x - width, precision, width, label='Precision')
plt.bar(x, recall, width, label='Recall')
plt.bar(x + width, f1, width, label='F1-score')
plt.xticks(x, labels, rotation=45)
plt.ylabel("Score")
plt.title("Precision, Recall, F1-score per label")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
import seaborn as sns

conf_matrices = multilabel_confusion_matrix(y_test, y_pred_bin)

for i, matrix in enumerate(conf_matrices):
    plt.figure(figsize=(3,3))
    sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix for Label {i}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.show()